# Clustering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# genero i dati:
N = 6
n_points = [20, 20, 20, 30, 30, 40]
noise_w = 0.3
# scelgo casualmente in [-10,10]x[-10,10] N centroidi a valori reali
centroids = np.random.rand(N, 2) * 20 - 10
# stabilisco delle varianze casuali per questi centroidi tra [0.5, 2]
variances = np.random.rand(N) * 1.5 + 0.5
# stabilisco il numero di punti per ogni cluster
# genero i punti
data = [np.random.randn(n, 2) * variances[i] + centroids[i] for i, n in enumerate(n_points)]

# inserisco del rumore: aggiungo dei punti uniformemente distribuiti in [-15,15]*[-15,15]
noise = np.random.rand(int(noise_w*sum(n_points)), 2) * 30 - 15

# plotto tutto
fig, ax = plt.subplots(figsize=(10, 10))
for i in range(N):
    ax.scatter(data[i][:, 0], data[i][:, 1], label='Cluster %d' % i)

ax.scatter(noise[:, 0], noise[:, 1], label='Noise', c='black', alpha=0.5)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


## KMeans algorithm

In [ ]:
# eseguo il classico KMeans
from sklearn.cluster import KMeans

fused_data = np.concatenate(data + [noise])
kmeans = KMeans(n_clusters=N, init='k-means++', max_iter=1000)
kmeans.fit(fused_data)
labels = kmeans.predict(fused_data)

# plotto i risultati:
# i centroidi sono indicati con una x e i dati con dei cerchi
# il colore di un cluster è dettato dal label stimato con kmeans
plt.figure(figsize=(10, 10))
for i in range(N):
    plt.scatter(fused_data[labels == i][:, 0], fused_data[labels == i][:, 1], label='Cluster %d' % i)
    plt.scatter(kmeans.cluster_centers_[i][0], kmeans.cluster_centers_[i][1], marker='x', s=100, c='black')

# la legenda la voglio fuori dal plot
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## Gaussian Mixture Clustering

In [ ]:
# eseguo il Gaussian Mixture Model
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

n_clusters = 6
gmm = GaussianMixture(n_components=n_clusters, covariance_type='full', init_params='kmeans', max_iter=1000, tol=1e-6)

gmm.fit(fused_data)
labels = gmm.predict(fused_data)

# plotto i risultati:
# i centroidi sono indicati con una x e i dati con dei cerchi
# il colore di un cluster è dettato dal label stimato con gmm
# indico con un'ellisse la covarianza del cluster
# indico il peso del cluster con un numero vicino al centroide
plt.figure(figsize=(10, 10))
for i in range(N):
    plt.scatter(fused_data[labels == i][:, 0], fused_data[labels == i][:, 1], label='Cluster %d' % i)
    plt.scatter(gmm.means_[i][0], gmm.means_[i][1], marker='x', s=100, c='black')
    # inserisco l'ellisse
    cov = gmm.covariances_[i]
    v, w = np.linalg.eigh(cov)
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180 * angle / np.pi
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    ell = Ellipse(xy=gmm.means_[i], width=v[0], height=v[1], angle=180 + angle, color='black', alpha=0.5)
    plt.gca().add_patch(ell)
    # peso del cluster
    plt.text(gmm.means_[i][0], gmm.means_[i][1], '%.2f' % gmm.weights_[i], fontsize=12)

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## FCM algorithm

In [ ]:
# eseguo la clusterizzazione FCM usando torch

# importo da matplotlib le linee
from matplotlib.lines import Line2D
import torch
# setto il device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# con kmeans stabilisco i centroidi iniziali
kmeans = KMeans(n_clusters=N, init='k-means++', max_iter=1000)
kmeans.fit(fused_data)
centroids = kmeans.cluster_centers_

# calcolo la matrice delle distanze a 2 a 2 tra dati e centroidi
data_torch = torch.tensor(fused_data, device=device)
centroids = torch.tensor(centroids, device=device)
m = 2

for i in range(10000):
    distances = torch.cdist(data_torch, centroids) ** (2/(m-1))
    # calcolo la matrice di membership
    U = 1 / (distances * torch.sum(1 / distances, dim=1, keepdim=True))

    # calcolo i nuovi centroidi
    Um = U ** m
    centroids = torch.matmul(Um.T, data_torch) / torch.sum(Um, dim=0, keepdim=True).T

# plotto i risultati:
# i centroidi sono indicati con una x e i dat9i con dei cerchi
# il colore di un cluster è dettato dal label stimato con fcm (argmax)
# il valore di tale collegamente è indicato con una linea tanto più chiara quanto più è basso
plt.figure(figsize=(10, 10))
labels = torch.argmax(U, dim=1)

# passo a numpy per poter usare le funzioni di matplotlib
labels = labels.cpu().numpy()
centroids = centroids.cpu().numpy()
U = U.cpu().numpy()
for i in range(N):
    plt.scatter(fused_data[labels==i][:,0], fused_data[labels==i][:,1], label='Cluster %d' % i)
    plt.scatter(centroids[i,0], centroids[i,1], marker='x', s=100, c='black')
for j in range(len(fused_data)):
    middle = 1./len(centroids)
    for i in range(N):
        if U[j,i] >= middle:
            line = Line2D(
                [fused_data[j,0], centroids[i,0]],
                [fused_data[j,1], centroids[i,1]],
                alpha=U[j,i],
                color='black',
            )
            plt.gca().add_line(line)
    
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()
    

# Cleaning and gray scale

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt

# leggo un'immagine non preprocessata
img = Image.open("data/db/cutted_set/Author2/Author2_0001_01.png").convert("L")

# mostro l'immagine a schermo
plt.imshow(img)
plt.show()

Studio la densità dei grigi nell'immagine

In [ ]:
import numpy as np

# la converto in un array numpy
img_array = np.array(img, dtype=np.float32)/256.0

# mostro un grafico con la densità in scala di grigi
# sull'ascisse un valore tra 0 e 1, sull'ordinata il numero di pixel con quel valore
plt.hist(img_array.flatten(), bins=256, color='black', density=True)
plt.show()

In [ ]:
# taglio i colori in base al loro valore rispetto un threshold
threshold = 0.25
img_array_cut = np.zeros_like(img_array)
img_array_cut[img_array < threshold] = 0
img_array_cut[img_array >= threshold] = 1

# mostro l'immagine a schermo
plt.imshow(img_array_cut, cmap='gray')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# eseguo una clusterizzazione kmeans su tutti i valori tra 0 e 1 con n_centroids
n_centroids = 8
img_flat = img_array.reshape(-1, 1)
kmeans = KMeans(n_clusters=n_centroids, init='k-means++', max_iter=1000)
kmeans.fit(img_flat)
labels = kmeans.predict(img_flat)
centroids = kmeans.cluster_centers_

# controllo che il numero di iterazioni non abbia superato max_iter
if kmeans.n_iter_ == kmeans.max_iter:
    print('KMeans non ha convergenza')

# riodino i centroidi e i labels in ordine crescente
order = np.argsort(centroids.flatten())
centroids = centroids[order]
new_labels = np.zeros_like(labels)
for i, o in enumerate(order):
    new_labels[labels == o] = i
labels = new_labels

# Normalize labels to range [0, 1]
normalized_labels = labels / labels.max()

# Create a colormap
cmap = plt.get_cmap('viridis')

# Plot histogram for each cluster
hist, bin_edges = np.histogram(img_array.flatten(), bins=256, density=False)
for i in range(n_centroids):
    cluster_data = img_flat[labels == i]
    cluster_hist, _ = np.histogram(
        cluster_data,
        bins=bin_edges,
        density=False
    )
    plt.hist(
        bin_edges[:-1],
        bins=bin_edges,
        weights=cluster_hist / hist.sum() * 256,
        color=cmap(i / n_centroids),
        alpha=0.5,
    )

# Add vertical lines for centroids
for c in centroids:
    plt.axvline(c, color='blue')

plt.show()

In [ ]:
# ricostruisco l'immagine originale usando i centroidi
img_reconstructed = centroids[labels].reshape(img_array.shape)

# mostro l'immagine ricostruita
plt.imshow(img_reconstructed, cmap='gray')
plt.show()

In [ ]:
# nell'immagine faccio sì che i pixel del cluster k-esimo siano colorati di rosso

# cluster scelto
for k in range(n_centroids):
    # creo un'immagine vuota
    img_colored = np.zeros(img_array.shape + (3,))
    # coloro i pixel del cluster k-esimo di rosso
    img_colored_flat = img_colored.reshape(-1, 3)
    img_colored_flat[labels == k] = [1, 0, 0]
    img_colored = img_colored_flat.reshape(img_array.shape + (3,))

    # riproduco l'immagine
    plt.imshow(img_colored)
    plt.title('Cluster %d' % k)
    plt.show()


In [ ]:
# normalizzo l'immagine affinché solo i claster da A a B siano visibili (con valori tra 0 e 1)

# scritte in 14-20/64

A = 0
B = 5
# cerco il colore del pixel dentro il cluster A di valore minimo
min_A = img_flat[labels == A].min()
max_B = img_flat[labels == B].max()
img_normalized = (img_array - min_A) / (max_B - min_A)
# taglio i valori < 0 a 0 e i valori > 1 a 1
img_normalized[img_normalized < 0] = 0
img_normalized[img_normalized > 1] = 1

# salvo l'immagine come png
img_normalized = Image.fromarray(np.uint8(img_normalized*255))
img_normalized.save('data/out/normalized.png')

# Distanza tra distribuzioni

In [ ]:
n_dimensions = 16
n_clusters = 2
n_latbox = 2
n_data = 128
n_test = 1000

import torch
import numpy as np
from sklearn.cluster import KMeans
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

results = []
time_start = time.time()
for _ in range(n_test):
    A = torch.randn(n_data, n_dimensions, device=device)
    B = torch.randn(n_data, n_dimensions, device=device) + torch.ones(n_dimensions, device=device) / (n_dimensions**0.5)
    fused_d = torch.cat([A, B])
    fused_d = fused_d.view(-1, fused_d.shape[-1])

    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=1000)
    fit = kmeans.fit(fused_d.cpu().numpy())
    centers = torch.from_numpy(fit.cluster_centers_).to(device=device)
    distances = torch.cdist(fused_d, centers)
    mu = torch.zeros(n_clusters, device=device)
    for i in range(n_clusters):
        mu[i] = torch.mean(distances[fit.labels_ == i, i])
    mu = (mu**0.5)
    p_A = torch.zeros(n_clusters, device=device)
    p_B = torch.zeros(n_clusters, device=device)
    for i in range(n_clusters):
        p_A[i] = np.sum(fit.labels_[:A.shape[0]] == i) / A.shape[0]
        p_B[i] = np.sum(fit.labels_[A.shape[0]:] == i) / B.shape[0]
    s = 0
    d = 0
    for i in range(n_clusters):
        s += (p_A[i] - p_B[i])**2 / (p_A[i] + p_B[i])**2 * mu[i]
        d += mu[i]
    s /= d
    D_and=0
    D_or=0
    for i in range(n_clusters):
        D_and += 1 if p_A[i] > 0 and p_B[i] > 0 else 0
        D_or += 1 if p_B[i] > 0 or p_B[i] > 0 else 0
    jaccard = (1 + D_and / D_or)**(-1)
    results.append(jaccard * s.cpu())
time_end = time.time()

print(np.mean(results))
print(np.var(results))
print((time_end - time_start)/n_test)
print()

results = []
time_start = time.time()
for _ in range(n_test):
    A = torch.randn(n_data, n_dimensions, device=device)
    B = torch.randn(n_data, n_dimensions, device=device) + torch.ones(n_dimensions, device=device) / (n_dimensions**0.5)
    fused_d = torch.cat([A, B])
    fused_d = fused_d.view(-1, fused_d.shape[-1])

    labels = torch.zeros(fused_d.shape[0], device='cuda', dtype=torch.int32)
    for i in range(fused_d.shape[1]):
        step = (torch.max(fused_d[:,i]) - torch.min(fused_d[:,i])) / n_latbox
        m = torch.min(fused_d[:,i])
        labels = labels*int(n_latbox) + torch.clamp(((fused_d[:,i] - m) // step).to(torch.int32), min=0, max=n_latbox - 1)
    
    # sort labels with counting sort
    countA: dict = {}
    for l in labels[:A.shape[0]]:
        if l in [k[0] for k in countA.items()]:
            countA[l.item()] += 1
        else:
            countA[l.item()] = 1
    
    countB: dict = {}
    for l in labels[A.shape[0]:]:
        if l in [k[0] for k in countB.items()]:
            countB[l.item()] += 1
        else:
            countB[l.item()] = 1
    
    A_keys = {k[0] for k in countA.items()}
    B_keys = {k[0] for k in countB.items()}

    s = 0
    all_keys = A_keys.union(B_keys)
    for i in all_keys:
        if i in A_keys:
            if i in B_keys:
                s += (countA[i]/A.shape[0] - countB[i]/B.shape[0])**2/(countA[i]/A.shape[0] + countB[i]/B.shape[0])**2
            else:
                s += 1
        elif i in B_keys:
            s += 1
    s /= (len(A_keys) + len(B_keys))
    results.append(s)
time_end = time.time()

print(np.mean(results))
print(np.var(results))
print((time_end - time_start)/n_test)
print()

In [ ]:
from sklearn.cluster import KMeans

# eseguo una clusterizzazione kmeans sulla fusione dei dati considerando i pesi
fused_d = torch.cat([A, B])
fused_w = torch.cat([w_A, w_B])
n_clusters = 64

fused_d = fused_d.view(-1, fused_d.shape[-1])
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=1000)

fit = kmeans.fit(fused_d.cpu().numpy(), sample_weight=fused_w.cpu().numpy())

centers = torch.from_numpy(fit.cluster_centers_).to(device=device)

# plot dei centroidi (o neri) assieme alla distribuzione dei dati (scatter blu)
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=fused_d[:, 0].cpu().numpy(), 
    y=fused_d[:, 1].cpu().numpy(),
    z=fused_d[:, 2].cpu().numpy(),
    mode='markers', 
    marker=dict(size=2, opacity=0.8, color='blue'), 
    name='Data'
))

fig.add_trace(go.Scatter3d(
    x=centers[:, 0].cpu().numpy(), 
    y=centers[:, 1].cpu().numpy(), 
    z=centers[:, 2].cpu().numpy(), 
    mode='markers', 
    marker=dict(size=3, color='black', symbol='x'), 
    name='Centroids'
))
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
# calcolo la distanza usando kmeans

# calcolo i pesi dei cluster come le medie quadratiche delle distanze dei dati dai rispettivi centroidi
distances = torch.cdist(fused_d, centers)
mu = torch.zeros(n_clusters, device=device)
for i in range(n_clusters):
    # uso i label per selezionare solo le distanze del cluster i-esimo
    mu[i] = torch.mean(distances[fit.labels_ == i, i])
mu = (mu**0.5)
# realizzo il grafico precedente mettendo però delle sfere di raggio w attorno ai centroidi
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=fused_d[:, 0].cpu().numpy(), 
    y=fused_d[:, 1].cpu().numpy(),
    z=fused_d[:, 2].cpu().numpy(),
    mode='markers', 
    marker=dict(size=2, opacity=0.8, color='blue'), 
    name='Data'
))
fig.add_trace(go.Scatter3d(
    x=centers[:, 0].cpu().numpy(), 
    y=centers[:, 1].cpu().numpy(), 
    z=centers[:, 2].cpu().numpy(), 
    mode='markers', 
    marker=dict(size=3, color='black', symbol='x'), 
    name='Centroids'
))
def make_sphere(center, radius):
    d = torch.tensor(np.pi/32, device=device)

    theta, phi = torch.meshgrid(torch.arange(0, torch.pi + d, d, device=device), torch.arange(0, 2 * torch.pi + d, d, device=device))
    # Convert to Cartesian coordinates
    x = torch.sin(theta) * torch.cos(phi)
    y = torch.sin(theta) * torch.sin(phi)
    z = torch.cos(theta)
    points = torch.vstack([x.ravel(), y.ravel(), z.ravel()])
    points = points * radius[:, None] + center[:, None]
    x, y, z = points
    return x,y,z

# assegno un colore ad ogni centroide
for i in range(n_clusters):
    x, y, z = make_sphere(centers[i], torch.tensor([mu[i], mu[i], mu[i]], device='cuda'))
    fig.add_trace(go.Mesh3d(x=x.cpu().numpy(), y=y.cpu().numpy(), z=z.cpu().numpy(), opacity=0.5, color='lightpink', alphahull=0))

fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
# calcolo il peso di ogni cluster rispettivamente per le due distribuzioni A e B
p_A = torch.zeros(n_clusters, device=device)
p_B = torch.zeros(n_clusters, device=device)
for i in range(n_clusters):
    p_A[i] = np.sum(fit.labels_[:A.shape[0]] == i) / A.shape[0]
    p_B[i] = np.sum(fit.labels_[A.shape[0]:] == i) / B.shape[0]

# calcolo la distanza tra le distribuzioni A e B:
s = 0
d = 0
for i in range(n_clusters):
    s += (p_A[i] - p_B[i])**2 / (p_A[i] + p_B[i])**2 * mu[i]
    d += mu[i]
s /= d

# jaccard index
# calcolo la quantità di centroidi che contengono elementi di A
D_and=0
D_or=0
for i in range(n_clusters):
    D_and += 1 if p_A[i] > 0 and p_B[i] > 0 else 0
    D_or += 1 if p_B[i] > 0 or p_B[i] > 0 else 0

jaccard = (1 + D_and / D_or)**(-1)

print(s)
print(jaccard)
print(jaccard * s)

In [ ]:
lateral_section = 6

import torch
import numpy as np
import plotly.graph_objects as go

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

results = []

for _ in range(1000):
    A_1 = torch.randn(4, 3, device=device) + torch.tensor([-2, 0, 1], device=device)
    A_2 = torch.randn(8, 3, device=device) * 2 + torch.tensor([4, 0, 0], device=device)
    A_3 = torch.randn(20, 3, device=device) * 0.5 + torch.tensor([0, 4, 0], device=device)
    A_noise = torch.rand(0, 3, device=device) * 20 - 10
    # concateno i dati
    A = torch.cat([A_1, A_2, A_3, A_noise], dim=0)
    w_A = torch.ones(A.shape[0], device=device)/A.shape[0]

    B_1 = torch.randn(12, 3, device=device) * torch.tensor([0.5, 2, -1], device=device) + torch.tensor([-1, 0, 0], device=device)
    B_2 = torch.randn(12, 3, device=device) + torch.tensor([4, 0, 0], device=device)
    B_3 = torch.randn(12, 3, device=device) * 0.5 + torch.tensor([0, -4, 0], device=device)
    B_noise = torch.rand(0, 3, device=device) * 20 - 10
    # concateno i dati
    B = torch.cat([B_1, B_2, B_3, B_noise], dim=0)
    w_B = torch.ones(B.shape[0], device=device)/B.shape[0]

    fused_d = torch.cat([A, B])

    # plotto i dati in 3d
    """fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=torch.cat([A[:, 0], B[:, 0]]).cpu().numpy(), 
        y=torch.cat([A[:, 1], B[:, 1]]).cpu().numpy(), 
        z=torch.cat([A[:, 2], B[:, 2]]).cpu().numpy(), 
        mode='markers', 
        marker=dict(
            size=3, 
            opacity=1.0, 
            color=['blue']*A.shape[0] + ['red']*B.shape[0]
        ), 
        name='A and B'
    ))
    # rendo l'immagine più grande
    fig.update_layout(width=800, height=800)
    fig.show()"""

    # calcolo la distanza usando però partizioni statiche
    labels = torch.zeros(fused_d.shape[0], device='cuda')
    for i in range(fused_d.shape[1]):
        step = (torch.max(fused_d[:,i]) - torch.min(fused_d[:,i])) / lateral_section
        m = torch.min(fused_d[:,i])
        labels = labels*lateral_section + (fused_d[:,i] - m) // step
    labels_A = labels[:A.shape[0]]
    labels_B = labels[A.shape[0]:]
    # calcolo la distanza nel modo classico
    s = torch.tensor([0.0], device='cuda')
    dA = torch.tensor([0.0], device='cuda')
    dB = torch.tensor([0.0], device='cuda')
    for i in range(lateral_section**3):
        f_A = torch.count_nonzero(labels_A == i) / labels_A.shape[0]
        f_B = torch.count_nonzero(labels_B == i) / labels_B.shape[0]
        if f_A > 0:
            dA += 1
            if f_B > 0:
                dB += 1
                s += (f_A - f_B)**2 / (f_A + f_B)**2
            else:
                s += 1
        elif f_B > 0:
            dB += 1
            s += 1
    results.append((s/(dA+dB)).item())

print(np.var(results))

In [ ]:
# uso centroids come initial centroids per fcm
iter = 10
fcm_centers = centers.clone()
for i in range(iter):
    distances = torch.cdist(fused_d, fcm_centers) ** 2
    
    # eseguo operazioni distinte su ogni riga di distances:
    # se la riga contiene uno 0 allora setto 1 dove ci sono 0 e 0 altrove
    # altrimenti calcolo la membership
    special_rows = torch.any(distances == 0, dim=1, keepdim=True)
    min_values = torch.min(distances, dim=1)[0]
    U = torch.where(special_rows, 
        distances != 0, # if row contains 0, set 1 where 0 and 0 elsewhere
        min_values[:,None] / distances, # else calculate membership
    )
    U = (U / U.sum(dim=1, keepdim=True))**2
    fcm_centers = torch.matmul(U.T, fused_d) / torch.sum(U, dim=0, keepdim=True).T

centers = fcm_centers.clone()
# plot dei centroidi (o neri) assieme alla distribuzione dei dati (scatter blu)
plt.figure(figsize=(5, 5))
plt.scatter(fused_d[:, 0].cpu(), fused_d[:, 1].cpu(), s=2, alpha=0.5, color='blue', label='Data')
plt.scatter(centers[:, 0].cpu(), centers[:, 1].cpu(), s=4, color='black', marker='o', label='Centroids')
# aggiungo le linee di membership, se la membership è superiore a 1/n_clusters inserisco una linea
indices = torch.where(U >= 1./n_clusters)
for i in range(len(indices[0])):
    plt.plot(
        [fused_d[indices[0][i], 0].cpu(), centers[indices[1][i], 0].cpu()],
        [fused_d[indices[0][i], 1].cpu(), centers[indices[1][i], 1].cpu()],
        color='black', alpha=U[indices[0][i], indices[1][i]].cpu().item()
    )
plt.legend()
plt.show()

In [ ]:
from matplotlib.patches import Circle
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap

# se si usa kmeans U[i,j] = 1 se il punto i è nel cluster j altrimenti 0
labels = kmeans.predict(fused_d.cpu().numpy())
U = torch.zeros(fused_d.shape[0], n_clusters, device=device)
for i in range(fused_d.shape[0]):
    U[i, labels[i]] = 1

# definisco la misura dei cluster
distances = torch.cdist(fused_d, centers) ** 2
mu = (torch.einsum('ij,ij->j', U, distances) / torch.sum(U, dim=0))**(fused_d.shape[1]/2)
# calcolo il peso di A e B sui cluster
omega_A = torch.mean(U[:A.shape[0], :], dim=0)  # A è il primo cluster
omega_B = torch.mean(U[A.shape[0]:, :], dim=0)  # B è il secondo cluster
# calcolo i rapporti tra i pesi su ogni cluster
r = torch.min(omega_A, omega_B) / torch.max(omega_A, omega_B)

# plotto lo spazio di misura usato usando mu(c)**(1/fused_d.shape[1]) come raggio del cluster c

# colormap
colors = [(1, 0, 0), (0, 0, 1), (0, 1, 0)]
n_bins = 256  # numero di intervalli nella colormap
cmap_name = 'membership_colormap'
my_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

fig, ax = plt.subplots(figsize=(12, 10))
ax.scatter(fused_d[:, 0].cpu(), fused_d[:, 1].cpu(), s=4, alpha=0.4, color='blue')
for i in range(n_clusters):
    s = (omega_B[i] - omega_A[i])/(omega_B[i] + omega_A[i])
    circle = Circle(
        centers[i].cpu(),
        mu[i].cpu()**(1/fused_d.shape[1]),
        color=my_cmap(s.cpu()),
        alpha=0.8)
    ax.add_patch(circle)

# inserisco la mappa di colori
# Crea manualmente la colorbar
norm = plt.Normalize(vmin=0, vmax=1)
sm = cm.ScalarMappable(cmap=my_cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)  # Associa la colorbar all'asse 'ax'
cbar.set_label('membership')
cbar.set_ticks([0, 1])  # Imposta i tick manualmente
cbar.set_ticklabels(['A', 'B'])  # Imposta le etichette dei tick

plt.show()

In [ ]:
# Indice di Jaccard
# computo D_A come la membership dei centroidi in A e D_B come la membership dei centroidi in B
D_A = torch.max(U[:A.shape[0], :], dim=0)[0]
D_B = torch.max(U[A.shape[0]:, :], dim=0)[0]
# computo la cardinalità dell'unione e dell'intersezione
card_inter = torch.sum(torch.min(D_A, D_B))
card_union = torch.sum(torch.max(D_A, D_B))
# computo l'indice di Jaccard
J = card_inter / card_union
print("Cardinalità di D_A: ", torch.sum(D_A))
print("Cardinalità di D_B: ", torch.sum(D_B))
print("Cardinalità dell'intersezione: ", card_inter)
print("Cardinalità dell'unione: ", card_union)
print("Indice di Jaccard: ", J)

In [ ]:
# calcoliamo la distanza tra A e B
integral = (((r - 1)/(r + 1))**2 @ mu)/(torch.sum(mu))
reguliser = (1+J)**(-1)
distance = integral * reguliser

print("Integrale: ", integral)
print("Regolarizzatore: ", reguliser)
print("Distanza tra A e B: ", distance)

In [ ]:
# plotto lo spazio di misura ottenuto:
# evidenzio i cluster con un grafico a barre
# ogni barra ha un'altezza proporzionale al peso del cluster e un ampiezza proporzionale alla misura
import matplotlib.pyplot as plt
from matplotlib.patches import Circle


plt.figure(figsize=(10, 10))

# aggiungo gli scatter fused_d in blu
plt.scatter(fused_d[:, 0].cpu(), fused_d[:, 1].cpu(), color='blue', alpha=0.5, s=2)

for i in range(n_clusters):
    # creo una sfera con raggio proporzionale alla misura
    # il suo colore dipende da weights ed è indicato con color
    circle = Circle(
        centers[i].cpu().numpy(),
        measure[i].item()**(1/fused_d.shape[1]),
        color=(1-weights[i].item(), 0, weights[i].item()),
        alpha=0.8,
    )
    # aggiungo il rettangolo al grafico
    plt.gca().add_patch(circle)


plt.xlim(-10, 10)
plt.ylim(-10, 10)

plt.show()

In [ ]:
# fit di A e B su siffatta clusterizzazione
labels_A = fit.predict(A.cpu().numpy().reshape(-1, fused_d.shape[1]))
labels_B = fit.predict(B.cpu().numpy().reshape(-1, fused_d.shape[1]))

# densità di probabilità di A e B sui cluster
p_A = torch.zeros(n_clusters, device=device)
p_B = torch.zeros(n_clusters, device=device)
for i in range(n_clusters):
    p_A[i] = torch.sum(w_A[labels_A == i])
    p_B[i] = torch.sum(w_B[labels_B == i])
p_A /= torch.sum(p_A)
p_B /= torch.sum(p_B)

In [ ]:
# plotto le densità
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm

# colormap
colors = [(1, 0, 0), (0.5, 0.5, 1), (0, 1, 0)]
n_bins = 256  # numero di intervalli nella colormap
cmap_name = 'membership_colormap'
my_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

fig, ax = plt.subplots()  # Crea una figura e un asse
for i in range(n_clusters):
    if measure[i] == 0:
        # metto una X blu per i cluster vuoti
        ax.scatter(centers[i,0].item(), centers[i,1].item(), marker='x', color='blue')
    else:
        # calcolo l'addendo senza elevare al quadrato
        s = ((p_A[i] - p_B[i])/(p_A[i] + p_B[i]))
        s = (s + 1)/2
        # creo un cerchio per il cluster i-esimo
        circ = Circle(
            centers[i].cpu(),
            measure[i].item()**(1/fused_d.shape[1]),
            color=my_cmap(s.cpu()),
            alpha=1.0,
        )
        # aggiungo il rettangolo al grafico
        ax.add_patch(circ)

# inserisco la mappa di colori
# Crea manualmente la colorbar
norm = plt.Normalize(vmin=0, vmax=1)
sm = cm.ScalarMappable(cmap=my_cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)  # Associa la colorbar all'asse 'ax'
cbar.set_label('membership')

plt.xlim(-6, 12)
plt.ylim(-9, 8)

plt.show()

In [ ]:
# ne calcolo la distanza
D_A = p_A > 0
D_B = p_B > 0

integral = torch.where(D_A | D_B, measure * ((p_A-p_B)/(p_A+p_B))**2, torch.zeros(n_clusters, device=device)).sum() / measure[D_A | D_B].sum()
Jaccard_index = measure[D_A & D_B].sum() / measure[D_A | D_B].sum()

distance = (1 + Jaccard_index)**(-1) * integral
print(distance.item())

## controprova senza clustering

In [ ]:
nodes_perdim = 32

# centri dei cluster sono una meshgrid
centers = torch.meshgrid([torch.linspace(-6, 12, nodes_perdim), torch.linspace(-9, 8, nodes_perdim)])
x_radius = 9 / nodes_perdim
y_radius = 8.5 / nodes_perdim

# la misura è unica e uguale per tutti quindi non serve
# calcolo le densità di probabilità di A e B sui cluster
p_A = torch.zeros((nodes_perdim,nodes_perdim), device=device)
p_B = torch.zeros((nodes_perdim,nodes_perdim), device=device)
for i in range(nodes_perdim):
    for j in range(nodes_perdim):
        # conto quanti punti di A e B sono nel cluster i-esimo, ossia nel range [centers[i]-radius, centers[i]+radius]
        p_A[i,j] = torch.sum(
            (A[:,0] >= centers[0][i,j]-x_radius) & (A[:,0] < centers[0][i,j]+x_radius) &
            (A[:,1] >= centers[1][i,j]-y_radius) & (A[:,1] < centers[1][i,j]+y_radius)
        )
        p_B[i,j] = torch.sum(
            (B[:,0] >= centers[0][i,j]-x_radius) & (B[:,0] < centers[0][i,j]+x_radius) &
            (B[:,1] >= centers[1][i,j]-y_radius) & (B[:,1] < centers[1][i,j]+y_radius)
        )
p_A /= torch.sum(p_A)
p_B /= torch.sum(p_B)

p_A = p_A.view(-1)
p_B = p_B.view(-1)

# calcolo le distanze
D_A = p_A > 0
D_B = p_B > 0
integral = torch.where(D_A | D_B, ((p_A-p_B)/(p_A+p_B))**2, torch.zeros(nodes_perdim*nodes_perdim, device=device)).sum() / torch.count_nonzero(D_A | D_B)
Jaccard_index = torch.count_nonzero(D_A & D_B) / torch.count_nonzero(D_A | D_B)

distance = (1 + Jaccard_index)**(-1) * integral
print(distance.item())

# FFT 2D

In [ ]:
import torch
import math

def my_map(X, Y):
    return 2*torch.cos(2*math.pi*(2*X + 3*Y) + 3) + 0.8*torch.cos(2*math.pi*(X + 5*Y) + 2) + torch.cos(2*math.pi*(7*X + 5*Y)) + torch.randn(X.shape)

# genero X,Y in una matrice 32x16 con valori da 0
X, Y = torch.meshgrid([torch.linspace(0, 1, 64), torch.linspace(0, 1, 32)])

Z = my_map(X,Y)

# calcolo la trasformata di fourier 2d con fft
Z_fft = torch.fft.fft2(Z)

# plotting
from matplotlib import pyplot as plt

# creo 2 plot disposti orizzontalmente
fig, ax = plt.subplots(1, 2, figsize=(10, 10))

# in ax[0] mostro la mappa Z
ax[0].imshow(Z.cpu().numpy(), cmap='viridis')

# in ax[1] mostro la mappa Z_fft
ax[1].imshow(torch.abs(Z_fft).cpu().numpy(), cmap='viridis')

plt.show()

In [ ]:
L = [{0}]
F = [{1}]

index = 0
for i in range(1,10000):
    for j in range(len(L)-1, -1, -1):
        # controllo se L[j-1] si interseca con F[index] se non si interseca siamo a cavallo
        if not (L[j] & F[index]) and not i in F[j]:
            # i due insiemi non si intersecano
            L[j].add(i)
            F[j].add(i+1)
            F[j].add(i-1)
            index = j
            break
    else:
        # non esiste alcun j valido quindi incremento
        L.append({i})
        F.append({i+1, i-1})
        index = -1

# trasformo in lista ordinata ogni insieme in L
for i in range(len(L)):
    L[i] = sorted(list(L[i]))

for l in L:
    print(l)

# PreProcessing

In [ ]:
# prendo due immagini e le preprocesso
from PIL import Image
from matplotlib import pyplot as plt
from source.packages import cleaner
import logging

cleaner.fft(
    logging.getLogger(),    
    "data/db/cutted_set/Author3",
    "data/.out",
    False,
    0.001,
    'cuda'
)

In [ ]:
# open image
img = Image.open("data/db/cutted_set/Author3/Author3_0001_01.png").convert("L")
# plot dell'immagine
plt.subplot(2,2,1)
plt.title("my image", cmap='gray')
plt.imshow(img)
plt.subplot(2,2,2)
plt.title("my image")
plt.imshow(img, cmap='gray')
plt.show()

# Compare Stability

In [5]:
import numpy as np
from source.packages import distance, clustering
import logging
import os
from sklearn.decomposition import PCA

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove all handlers associated with the logger object
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

n_tiles = 6
fcm_tollerance = 10**-8
final_dim = 10
n_clusters = 128

# elenco tutte le opere
my_list = []
for root, dirs, files in os.walk(r"./data/.out/synthetized"):
    for file in files:
        if file.endswith(".png"):
            my_list.append(os.path.join(root, file))

# calcolo per 10 volte la distanza tra le due sintesi
dist_list = []
work_1 = np.random.choice(my_list)
work_2 = np.random.choice(my_list)
for i in range(10):

    # open synthesis
    with open(work_1, "br") as f:
        values = f.read()
    synth_1 = (
        np.frombuffer(values, dtype=np.uint8)
        .reshape(-1, n_tiles * n_tiles)
        .astype(np.float32)
        / 255.0
    )
    # open synthesis
    with open(work_2, "br") as f:
        values = f.read()
    synth_2 = (
        np.frombuffer(values, dtype=np.uint8)
        .reshape(-1, n_tiles * n_tiles)
        .astype(np.float32)
        / 255.0
    )
    synth_merge = np.vstack((synth_1, synth_2))

    # eseguo una pca
    pca = PCA(n_components=final_dim)
    synth_merge = pca.fit_transform(synth_merge)

    # salvo la sintesi merge in un file temporaneo come float32 binario
    with open(r"./temp/synth_merge", "bw") as f:
        f.write(synth_merge.tobytes())
    # estraggo un campione di n_clusters righe da synth_merge
    synth_sample = np.random.choice(
        synth_merge.shape[0], n_clusters, replace=False
    )
    synth_sample = synth_merge[synth_sample]
    # aggiungo del noise
    synth_sample += np.random.normal(0, 0.01, synth_sample.shape)
    # salvo il campione in un file temporaneo come float32 binario (i centroidi iniziali)
    with open(r"./temp/synth_sample", "bw") as f:
        f.write(synth_sample.tobytes())

    # costruisco il campione dei pesi, tutti uguali per i rispettivi synth
    synth_weights = np.ones((synth_merge.shape[0]), dtype=np.float32)
    # mi chiedo quale sia la sintesi con più dati
    synth_weights[: synth_1.shape[0]] = (
        np.ones((synth_1.shape[0]), dtype=np.float32)
        / synth_1.shape[0]
    )
    synth_weights[synth_1.shape[0] :] = (
        np.ones((synth_2.shape[0]), dtype=np.float32)
        / synth_2.shape[0]
    )
    # salvo i pesi in un file temporaneo come float32 binario
    with open(r"./temp/synth_weights", "bw") as f:
        f.write(synth_weights.tobytes())

    # libero la ram (fondamentale per evitare memory error)
    del values
    del synth_1
    del synth_2
    del synth_merge  # ora presente in ./temp/synth_merge
    del synth_sample  # ora presente in ./temp/synth_sample
    del synth_weights  # ora presente in ./temp/synth_weights

    # eseguo il clustering fcm
    try:
        clustering.fcm(
            logger,
            r"./temp/synth_merge",
            r"./temp/synth_weights",
            r"./temp/synth_sample",
            r"./temp/centroids",
            final_dim,
            fcm_tollerance,
            r"./logs/fcm.log",
        )
    except SyntaxError:
        logger.critical("Implementation error!")
        exit()
    except ValueError:
        logger.error("Unvalid inputs")
        exit()
    except Exception:
        logger.error("Unexpected error")
        exit()

    # uso i centroidi in "./temp/centroids" per calcolare la distanza tra i due synth
    # load data
    with open(work_1, "br") as f:
        values = f.read()
    synth_1 = (
        np.frombuffer(values, dtype=np.uint8)
        .reshape(-1, n_tiles * n_tiles)
        .astype(np.float32)
        / 255.0
    )
    with open(work_2, "br") as f:
        values = f.read()
    synth_2 = (
        np.frombuffer(values, dtype=np.uint8)
        .reshape(-1, n_tiles * n_tiles)
        .astype(np.float32)
        / 255.0
    )
    # riduco la dimensionalità usando la stessa pca precedente
    synth_1 = pca.transform(synth_1)
    synth_2 = pca.transform(synth_2)

    weights_1 = (
        np.ones((synth_1.shape[0]), dtype=np.float32)
        / synth_1.shape[0]
    )
    weights_2 = (
        np.ones((synth_2.shape[0]), dtype=np.float32)
        / synth_2.shape[0]
    )
    with open(r"./temp/centroids", "br") as f:
        values = f.read()
    centroids = np.frombuffer(values, dtype=np.float32).reshape(
        n_clusters, final_dim
    )
    # compute distance
    try:
        dist = distance.compute_distance(
            logger,
            synth_1,
            synth_2,
            weights_1,
            weights_2,
            centroids,
        )
    except SyntaxError:
        logger.critical("Implementation error!")
        exit()
    except ValueError:
        logger.error("Unvalid inputs")
        exit()
    except Exception:
        logger.error("Unexpected error")
        exit()
    else:
        logger.info(f"Computed distance between {work_1}, {work_2}: {dist}")

2024-12-22 22:34:14,293 - root - INFO - fuzzy parameters: 3.612734079360962 shared centroids, 3.702620029449463 total centroids
2024-12-22 22:34:14,294 - root - INFO - mean integral: 0.014932204969227314
2024-12-22 22:34:14,294 - root - INFO - distance: 0.0075578405521810055
2024-12-22 22:34:14,295 - root - INFO - Computed distance between ./data/.out/synthetized/Author1/Author1_0040_09.png, ./data/.out/synthetized/Author3/Author3_0008_03.png: 0.0075578405521810055
2024-12-22 22:34:17,710 - root - INFO - fuzzy parameters: 4.352913856506348 shared centroids, 4.465834617614746 total centroids
2024-12-22 22:34:17,711 - root - INFO - mean integral: 0.014846491627395153
2024-12-22 22:34:17,712 - root - INFO - distance: 0.007518297526985407
2024-12-22 22:34:17,712 - root - INFO - Computed distance between ./data/.out/synthetized/Author1/Author1_0040_09.png, ./data/.out/synthetized/Author3/Author3_0008_03.png: 0.007518297526985407
2024-12-22 22:34:22,351 - root - INFO - fuzzy parameters: 3.62

In [ ]:
np.std(dist_list)

# Comparing

In [ ]:
# Genero i dati
import torch
import numpy as np
from source.packages import clustering
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

n_dimensions = 2

# genero il set di dati di A in R2
data_A1 = torch.randn(5, n_dimensions) / 2 + (torch.rand(n_dimensions) - 0.5)*20
weights_A1 = torch.rand((data_A1.shape[0],))*0.8+0.6
data_A2 = torch.randn(5, n_dimensions) + (torch.rand(n_dimensions) - 0.5)*10
weights_A2 = torch.rand((data_A2.shape[0],))*0.8+0.6
data_A3 = torch.randn(5, n_dimensions) * 2 + (torch.rand(n_dimensions) - 0.5)*20
weights_A3 = torch.rand((data_A3.shape[0],))*0.8+0.6
data_A4 = torch.randn(15, n_dimensions) * 2 + (torch.rand(n_dimensions) - 0.5)*20
weights_A4 = torch.rand((data_A4.shape[0],))*0.8+0.6
data_A5 = torch.randn(15, n_dimensions) / 2 + (torch.rand(n_dimensions) - 0.5)*20
weights_A5 = torch.rand((data_A5.shape[0],))*0.8+0.6
data_A6 = torch.randn(25, n_dimensions) + (torch.rand(n_dimensions) - 0.5)*20
weights_A6 = torch.rand((data_A6.shape[0],))*0.8+0.6

# genero il set di dati di B in R3
data_B1 = torch.randn(5, n_dimensions) / 2 + (torch.rand(n_dimensions) - 0.5)*20 
weights_B1 = torch.rand((data_B1.shape[0],))*0.8+0.6
data_B2 = torch.randn(5, n_dimensions) + (torch.rand(n_dimensions) - 0.5)*20
weights_B2 = torch.rand((data_B2.shape[0],))*0.8+0.6
data_B3 = torch.randn(5, n_dimensions) * 2 + (torch.rand(n_dimensions) - 0.5)*20
weights_B3 = torch.rand((data_B3.shape[0],))*0.8+0.6
data_B4 = torch.randn(15, n_dimensions) / 2 + (torch.rand(n_dimensions) - 0.5)*20
weights_B4 = torch.rand((data_B4.shape[0],))*0.8+0.6
data_B5 = torch.randn(15, n_dimensions) * 2 + (torch.rand(n_dimensions) - 0.5)*20
weights_B5 = torch.rand((data_B5.shape[0],))*0.8+0.6
data_B6 = torch.randn(25, n_dimensions) + (torch.rand(n_dimensions) - 0.5)*20
weights_B6 = torch.rand((data_B6.shape[0],))*0.8+0.6

# aggiungo noise
noise_A = (torch.rand((5, n_dimensions)) - 0.5)*20
nweights_A = torch.ones((noise_A.shape[0],))
noise_B = (torch.rand((5, n_dimensions)) - 0.5)*20
nweights_B = torch.ones((noise_B.shape[0],))

# unisco i dati
data_A = torch.vstack([data_A1, data_A2, data_A3, data_A4, data_A5, data_A6, noise_A]).to(torch.float32)
weights_A = torch.concatenate([weights_A1, weights_A2, weights_A3, weights_A4, weights_A5, weights_A6, nweights_A]).to(torch.float32)
data_B = torch.vstack([data_B1, data_B2, data_B3, data_B4, data_B5, data_B6, noise_B]).to(torch.float32)
weights_B = torch.concatenate([weights_B1, weights_B2, weights_B3, weights_B4, weights_B5, weights_B6, nweights_B]).to(torch.float32)

# ottengo i dati uniti
data = torch.vstack([data_A, data_B])
weights = torch.concatenate([weights_A, weights_B])

print(data.shape, data.dtype)
print(weights.shape, weights.dtype)

In [ ]:
# plot data
from plotly import graph_objects as go

# scatter di data_A1, uso weights_A1 per la dimensione dei punti
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data_A[:, 0].cpu().numpy(),
    y=data_A[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_A.cpu().numpy()*10, color='red'),
    name='Data A',
    opacity=1.0
))

fig.add_trace(go.Scatter(
    x=data_B[:, 0].cpu().numpy(),
    y=data_B[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_B.cpu().numpy()*10, color='blue'),
    name='Data B',
    opacity=1.0
))

# le proporzioni della griglia devono essere 1:1
fig.update_layout(
    width=800,
    height=800,
    xaxis=dict(scaleanchor="y", scaleratio=1),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)
fig.show()

In [ ]:
# calcolo il clustering fcm
n_centroids = 15

with open(r"./temp/data", "bw") as f:
    f.write(data.cpu().numpy().tobytes())

with open(r"./temp/weights", "bw") as f:
    f.write(weights.cpu().numpy().tobytes())

init_centroids = torch.rand(n_centroids,n_dimensions).to(torch.float32)
with open(r"./temp/centroids", "bw") as f:
    f.write(init_centroids.cpu().numpy().tobytes())

with open(r"./temp/output", "bw") as f:
    pass

clustering.fcm(
    logger,
    r"./temp/data",
    r"./temp/weights",
    r"./temp/centroids",
    r"./temp/output",
    n_dimensions,
    10**-9,
    r"./logs/fcm.log",
)

with open(r"./temp/output", "br") as f:
    buf = f.read()
    centroids = torch.from_numpy(np.frombuffer(buf, dtype=np.float32)).reshape(-1,data.shape[1]).to(torch.float32)

# compute distances
d_distances = torch.cdist(data, centroids, p=2)**2
# compute mu
mu = 1 / (d_distances * torch.sum(1 / d_distances, dim=1, keepdim=True))
mu[torch.isnan(mu)] = 1
mu = mu / torch.sum(mu, dim=1, keepdim=True)

# compute weights
p = torch.einsum('ij, i -> j', mu, weights) / torch.sum(weights, dim=0)
# compute sigma
s = torch.einsum('ij, i -> j', mu**2, weights) / torch.einsum('ij, i -> j', mu, weights)
# compute E
E = torch.einsum('ij, i -> j', mu**2 * d_distances, weights) / torch.einsum('ij, i -> j', mu**2, weights)

# compute mu of cluster
measure = s * E ** (n_dimensions/2)

# show data with centroids
fig = go.Figure()
# make a dict with f"centroid {j}":mu[i,j]
myDict = {f"centroid {j}":mu[:,j].cpu().numpy() for j in range(n_centroids)}
fig.add_trace(go.Scatter(

    x=data_A[:, 0].cpu().numpy(),
    y=data_A[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_A.cpu().numpy()*10, color='red'),
    name='Data A',
    opacity=1.0,
))
fig.add_trace(go.Scatter(
    x=data_B[:, 0].cpu().numpy(),
    y=data_B[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_B.cpu().numpy()*10, color='blue'),
    name='Data B',
    opacity=1.0
))
fig.add_trace(go.Scatter(
    x=centroids[:, 0].cpu().numpy(),
    y=centroids[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=10, color='black', symbol='x'),
    name='Centroids',
    opacity=1.0
))
# aggiungo delle circonferenze di raggio measure**(1/n_dimensions)
for i in range(n_centroids):
    fig.add_shape(
        type='circle',
        xref='x',
        yref='y',
        x0=centroids[i, 0].item() - measure[i].item()**(1/n_dimensions),
        y0=centroids[i, 1].item() - measure[i].item()**(1/n_dimensions),
        x1=centroids[i, 0].item() + measure[i].item()**(1/n_dimensions),
        y1=centroids[i, 1].item() + measure[i].item()**(1/n_dimensions),
        line=dict(color='black', width=1)
    )

# voglio l'immagine grande
fig.update_layout(
    width=800,
    height=800,
    xaxis=dict(scaleanchor="y", scaleratio=1),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)
fig.show()

from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# calcolo la distanza tra A e B
p_A = torch.einsum('ij, i -> j', mu[:data_A.shape[0], :], weights[:data_A.shape[0]]) / torch.sum(weights[:data_A.shape[0]])
p_B = torch.einsum('ij, i -> j', mu[data_A.shape[0]:, :], weights[data_A.shape[0]:]) / torch.sum(weights[data_A.shape[0]:])
value = (p_A - p_B) / (p_A + p_B)

# calcolo la media integrale
integral = torch.sum(measure * value**2) / torch.sum(measure)

# calcolo la cardinalità di |D_A n D_B| e |D_A u D_B|
c_A = torch.max(mu[:data_A.shape[0], :], dim=0)[0]  # esistenza del cluster nel dizionario di A
c_B = torch.max(mu[data_A.shape[0]:, :], dim=0)[0]  # esistenza del cluster nel dizionario di B
card_inter = torch.sum(torch.minimum(c_A, c_B))
card_union = torch.sum(torch.maximum(c_A, c_B))
Jaccard_index = card_inter / card_union

distance = (1 + Jaccard_index)**(-1) * integral
print(f"{distance=}")
print(f"\t{Jaccard_index=}")
print(f"\t\t{card_inter=}", f"{card_union=}")
print(f"\t\t\t{c_A=}")
print(f"\t\t\t{c_B=}")
print(f"\t{integral=}")
print(f"\t\t{value=}")
print(f"\t\t\t{p_A=}")
print(f"\t\t\t{p_B=}")


# Plot dei cluster:
colors = [(0, 0, 1), (1, 0, 0)]
n_bins = 256  # numero di intervalli nella colormap
cmap_name = 'membership_colormap'
my_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
fig, ax = plt.subplots(figsize=(5, 4))
# add points
plt.scatter(data_A.cpu().numpy()[:,0], data_A.cpu().numpy()[:,1], weights_A.cpu().numpy(), c='red')
plt.scatter(data_B.cpu().numpy()[:,0], data_B.cpu().numpy()[:,1], weights_B.cpu().numpy(), c='blue')
for i in range(n_centroids):
    s = (value[i] + 1) / 2
    circle = plt.Circle(
        centroids[i].cpu(),
        measure[i].item()**(1/n_dimensions),
        color=my_cmap(s.item()),
        alpha=0.8
    )
    ax.add_artist(circle)
norm = plt.Normalize(vmin=-1, vmax=1)
sm = plt.cm.ScalarMappable(cmap=my_cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)  # Associa la colorbar all'asse 'ax'
cbar.set_label('integrand')
cbar.set_ticks([-1, 0, 1])  # Imposta i tick manualmente
cbar.set_ticklabels(['B', '0', 'A'])  # Imposta le etichette dei tick
plt.xlim(-20, 20)
plt.ylim(-20, 20)
plt.savefig('./data/fcm_comparison.png', dpi=600)
plt.show()

In [ ]:
# using kmeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=n_centroids)
kmeans.fit(data.cpu().numpy(), sample_weight=weights.cpu().numpy())
centroids = torch.from_numpy(kmeans.cluster_centers_).to(torch.float32)

# compute distances
d_distances = torch.cdist(data, centroids, p=2)**2
# compute mu: m[i,j] = 1 if label is 1 0 otherwise
mu = torch.zeros(data.shape[0], n_centroids)
for i in range(data.shape[0]):
    mu[i, kmeans.labels_[i]] = 1

# compute weights
p = torch.einsum('ij, i -> j', mu, weights) / torch.sum(weights, dim=0)
# compute sigma
s = torch.einsum('ij, i -> j', mu**2, weights) / torch.einsum('ij, i -> j', mu, weights)
# compute E
E = torch.einsum('ij, i -> j', mu**2 * d_distances, weights) / torch.einsum('ij, i -> j', mu**2, weights)

# compute mu of cluster
measure = s * E ** (n_dimensions/2)

fig = go.Figure()
# make a dict with f"centroid {j}":mu[i,j]
myDict = {f"centroid {j}":mu[:,j].cpu().numpy() for j in range(n_centroids)}
fig.add_trace(go.Scatter(
    x=data_A[:, 0].cpu().numpy(),
    y=data_A[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_A.cpu().numpy()*10, color='red'),
    name='Data A',
    opacity=1.0,
))
fig.add_trace(go.Scatter(
    x=data_B[:, 0].cpu().numpy(),
    y=data_B[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_B.cpu().numpy()*10, color='blue'),
    name='Data B',
    opacity=1.0
))
fig.add_trace(go.Scatter(
    x=centroids[:, 0].cpu().numpy(),
    y=centroids[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=10, color='black', symbol='x'),
    name='Centroids',
    opacity=1.0
))
# aggiungo delle circonferenze di raggio measure**(1/n_dimensions)
for i in range(n_centroids):
    fig.add_shape(
        type='circle',
        xref='x',
        yref='y',
        x0=centroids[i, 0].item() - measure[i].item()**(1/n_dimensions),
        y0=centroids[i, 1].item() - measure[i].item()**(1/n_dimensions),
        x1=centroids[i, 0].item() + measure[i].item()**(1/n_dimensions),
        y1=centroids[i, 1].item() + measure[i].item()**(1/n_dimensions),
        line=dict(color='black', width=1)
    )

# voglio l'immagine grande
fig.update_layout(
    width=800,
    height=800,
    xaxis=dict(scaleanchor="y", scaleratio=1),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)
fig.show()

# calcolo la distanza tra A e B
p_A = torch.einsum('ij, i -> j', mu[:data_A.shape[0], :], weights[:data_A.shape[0]]) / torch.sum(weights[:data_A.shape[0]])
p_B = torch.einsum('ij, i -> j', mu[data_A.shape[0]:, :], weights[data_A.shape[0]:]) / torch.sum(weights[data_A.shape[0]:])
value = (p_A - p_B) / (p_A + p_B)

# calcolo la media integrale
integral = torch.sum(measure * value**2) / torch.sum(measure)

# calcolo la cardinalità di |D_A n D_B| e |D_A u D_B|
c_A = torch.max(mu[:data_A.shape[0], :], dim=0)[0]  # esistenza del cluster nel dizionario di A
c_B = torch.max(mu[data_A.shape[0]:, :], dim=0)[0]  # esistenza del cluster nel dizionario di B
card_inter = torch.sum(torch.minimum(c_A, c_B))
card_union = torch.sum(torch.maximum(c_A, c_B))
Jaccard_index = card_inter / card_union

distance = (1 + Jaccard_index)**(-1) * integral
print(f"{distance=}")
print(f"\t{Jaccard_index=}")
print(f"\t\t{card_inter=}", f"{card_union=}")
print(f"\t\t\t{c_A=}")
print(f"\t\t\t{c_B=}")
print(f"\t{integral=}")
print(f"\t\t{value=}")
print(f"\t\t\t{p_A=}")
print(f"\t\t\t{p_B=}")

# Plot dei cluster:
colors = [(0, 0, 1), (1, 0, 0)]
n_bins = 256  # numero di intervalli nella colormap
cmap_name = 'membership_colormap'
my_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
fig, ax = plt.subplots(figsize=(5, 4))
# add points
plt.scatter(data_A.cpu().numpy()[:,0], data_A.cpu().numpy()[:,1], weights_A.cpu().numpy(), c='red')
plt.scatter(data_B.cpu().numpy()[:,0], data_B.cpu().numpy()[:,1], weights_B.cpu().numpy(), c='blue')
for i in range(n_centroids):
    s = (value[i] + 1) / 2
    circle = plt.Circle(
        centroids[i].cpu(),
        measure[i].item()**(1/n_dimensions),
        color=my_cmap(s.item()),
        alpha=0.8
    )
    ax.add_artist(circle)
norm = plt.Normalize(vmin=-1, vmax=1)
sm = plt.cm.ScalarMappable(cmap=my_cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)  # Associa la colorbar all'asse 'ax'
cbar.set_label('integrand')
cbar.set_ticks([-1, 0, 1])  # Imposta i tick manualmente
cbar.set_ticklabels(['B', '0', 'A'])  # Imposta le etichette dei tick
plt.xlim(-20, 20)
plt.ylim(-20, 20)
plt.savefig('./data/kmeans_comparison.png', dpi=600)
plt.show()

In [ ]:
# uso lo box per clusterizzare i dati: uso i 4 quadranti
# calcolo i centroidi
centroids = torch.zeros(n_centroids, n_dimensions)
for i in range(n_dimensions):
    for j in range(n_dimensions):
        centroids[i*n_dimensions+j] = torch.tensor([20*(i-0.5), 20*(j-0.5)])

# i label sono stabiliti in base al segno dei dati
mu = torch.zeros(data.shape[0], n_centroids)
for i in range(data.shape[0]):
    if data[i,0] < 0 and data[i,1] < 0:
        mu[i,0] = 1
    elif data[i, 0] < 0:
        mu[i,1] = 1
    elif data[i, 1] < 0:
        mu[i,2] = 1
    else:
        mu[i,3] = 1

measure = torch.ones(n_centroids)

fig = go.Figure()
# make a dict with f"centroid {j}":mu[i,j]
myDict = {f"centroid {j}":mu[:,j].cpu().numpy() for j in range(n_centroids)}
fig.add_trace(go.Scatter(
    x=data_A[:, 0].cpu().numpy(),
    y=data_A[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_A.cpu().numpy()*10, color='red'),
    name='Data A',
    opacity=1.0,
))
fig.add_trace(go.Scatter(
    x=data_B[:, 0].cpu().numpy(),
    y=data_B[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=weights_B.cpu().numpy()*10, color='blue'),
    name='Data B',
    opacity=1.0
))
fig.add_trace(go.Scatter(
    x=centroids[:, 0].cpu().numpy(),
    y=centroids[:, 1].cpu().numpy(),
    mode='markers',
    marker=dict(size=10, color='black', symbol='x'),
    name='Centroids',
    opacity=1.0
))
# aggiungo una linea nera orizzontale da -20 a 20 e verticale da -20 a 20
fig.add_shape(
    type='line',
    x0=-20,
    y0=0,
    x1=20,
    y1=0,
    line=dict(color='black', width=2)
)
fig.add_shape(
    type='line',
    x0=0,
    y0=-20,
    x1=0,
    y1=20,
    line=dict(color='black', width=2)
)

# voglio l'immagine grande
fig.update_layout(
    width=800,
    height=800,
    xaxis=dict(scaleanchor="y", scaleratio=1),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)
fig.show()

# calcolo la distanza tra A e B
p_A = torch.einsum('ij, i -> j', mu[:data_A.shape[0], :], weights[:data_A.shape[0]]) / torch.sum(weights[:data_A.shape[0]])
p_B = torch.einsum('ij, i -> j', mu[data_A.shape[0]:, :], weights[data_A.shape[0]:]) / torch.sum(weights[data_A.shape[0]:])
value = (p_A - p_B) / (p_A + p_B)

# calcolo la media integrale
integral = torch.sum(measure * value**2) / torch.sum(measure)

# calcolo la cardinalità di |D_A n D_B| e |D_A u D_B|
c_A = torch.max(mu[:data_A.shape[0], :], dim=0)[0]  # esistenza del cluster nel dizionario di A
c_B = torch.max(mu[data_A.shape[0]:, :], dim=0)[0]  # esistenza del cluster nel dizionario di B
card_inter = torch.sum(torch.minimum(c_A, c_B))
card_union = torch.sum(torch.maximum(c_A, c_B))
Jaccard_index = card_inter / card_union

distance = (1 + Jaccard_index)**(-1) * integral
print(f"{distance=}")
print(f"\t{Jaccard_index=}")
print(f"\t\t{card_inter=}", f"{card_union=}")
print(f"\t\t\t{c_A=}")
print(f"\t\t\t{c_B=}")
print(f"\t{integral=}")
print(f"\t\t{value=}")
print(f"\t\t\t{p_A=}")
print(f"\t\t\t{p_B=}")

# Plot dei cluster:
colors = [(0, 0, 1), (1, 0, 0)]
n_bins = 256  # numero di intervalli nella colormap
cmap_name = 'membership_colormap'
my_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
fig, ax = plt.subplots(figsize=(5, 4))
# add points
plt.scatter(data_A.cpu().numpy()[:,0], data_A.cpu().numpy()[:,1], weights_A.cpu().numpy(), c='red')
plt.scatter(data_B.cpu().numpy()[:,0], data_B.cpu().numpy()[:,1], weights_B.cpu().numpy(), c='blue')
for i in range(n_centroids):
    s = (value[i] + 1) / 2
    rectangle = plt.Rectangle(
        (centroids[i, 0].cpu() - 10, centroids[i, 1].cpu() - 10),
        20, 20,
        color=my_cmap(s.item()),
        alpha=0.8
    )
    ax.add_artist(rectangle)
norm = plt.Normalize(vmin=-1, vmax=1)
sm = plt.cm.ScalarMappable(cmap=my_cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)  # Associa la colorbar all'asse 'ax'
cbar.set_label('integrand')
cbar.set_ticks([-1, 0, 1])  # Imposta i tick manualmente
cbar.set_ticklabels(['B', '0', 'A'])  # Imposta le etichette dei tick
plt.xlim(-20, 20)
plt.ylim(-20, 20)
plt.savefig('./data/box_comparison.png', dpi=600)
plt.show()


# Stability comparison

In [ ]:
# genero i dati:
import torch
import numpy as np
from source.packages import clustering
from sklearn.cluster import KMeans
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

n_dimensions = 16
n_data = 500
n_centroids = 8
noise_intensity = 0.2

data_A = torch.randn(int(n_data * (1 - noise_intensity)), n_dimensions) + torch.ones(n_dimensions)
data_B = torch.randn(int(n_data * (1 - noise_intensity)), n_dimensions) - torch.ones(n_dimensions)
weights_A = torch.rand((data_A.shape[0],))*0.8+0.6
weights_B = torch.rand((data_B.shape[0],))*0.8+0.6
noise_A = (torch.rand(int(n_data * noise_intensity), n_dimensions) - 0.5)*10
noise_B = (torch.rand(int(n_data * noise_intensity), n_dimensions) - 0.5)*10
weights_noise_A = torch.ones((noise_A.shape[0],))
weights_noise_B = torch.ones((noise_B.shape[0],))

data_A = torch.vstack([data_A, noise_A]).to(torch.float32)
data_B = torch.vstack([data_B, noise_B]).to(torch.float32)
weights_A = torch.cat([weights_A, weights_noise_A])
weights_B = torch.cat([weights_B, weights_noise_B])
data = torch.vstack([data_A, data_B])
weights = torch.cat([weights_A, weights_B])

# normalizzo data in -1,1
data -= data.min(dim=0, keepdim=True)[0]
data /= data.norm(dim=1).max()

# clustering con fcm
with open(r"./temp/data", "bw") as f:
    f.write(data.cpu().numpy().tobytes())
with open(r"./temp/weights", "bw") as f:
    f.write(weights.cpu().numpy().tobytes())
init_centroids = torch.rand(n_centroids,n_dimensions).to(torch.float32)
with open(r"./temp/centroids", "bw") as f:
    f.write(init_centroids.cpu().numpy().tobytes())
with open(r"./temp/output", "bw") as f:
    pass
clustering.fcm(
    logger,
    r"./temp/data",
    r"./temp/weights",
    r"./temp/centroids",
    r"./temp/output",
    n_dimensions,
    10**-9,
    r"./logs/fcm.log",
)
with open(r"./temp/output", "br") as f:
    buf = f.read()
    centroids = torch.from_numpy(np.frombuffer(buf, dtype=np.float32)).reshape(-1,data.shape[1]).to(torch.float32)
d_distances = torch.cdist(data, centroids, p=2)**2
mu = 1 / (d_distances * torch.sum(1 / d_distances, dim=1, keepdim=True))
mu[torch.isnan(mu)] = 1
mu = mu / torch.sum(mu, dim=1, keepdim=True)
p = torch.einsum('ij, i -> j', mu, weights) / torch.sum(weights, dim=0)
s = torch.einsum('ij, i -> j', mu**2, weights) / torch.einsum('ij, i -> j', mu, weights)
E = torch.einsum('ij, i -> j', mu**2 * d_distances, weights) / torch.einsum('ij, i -> j', mu**2, weights)
measure = s * E ** (n_dimensions/2)
p_A = torch.einsum('ij, i -> j', mu[:data_A.shape[0], :], weights[:data_A.shape[0]]) / torch.sum(weights[:data_A.shape[0]])
p_B = torch.einsum('ij, i -> j', mu[data_A.shape[0]:, :], weights[data_A.shape[0]:]) / torch.sum(weights[data_A.shape[0]:])
value = (p_A - p_B) / (p_A + p_B)
integral = torch.sum(measure * value**2) / torch.sum(measure)
c_A = torch.max(mu[:data_A.shape[0], :], dim=0)[0]
c_B = torch.max(mu[data_A.shape[0]:, :], dim=0)[0]
card_inter = torch.sum(torch.minimum(c_A, c_B))
card_union = torch.sum(torch.maximum(c_A, c_B))
Jaccard_index = card_inter / card_union
distance = (1 + Jaccard_index)**(-1) * integral
print(f"{distance=}")
print(f"\t{Jaccard_index=}")
print(f"\t\t{card_inter=}", f"{card_union=}")
print(f"\t\t\t{c_A=}")
print(f"\t\t\t{c_B=}")
print(f"\t{integral=}")
print(f"\t\t{value=}")
print(f"\t\t\t{p_A=}")
print(f"\t\t\t{p_B=}")
print()
print()

# kmeans
kmeans = KMeans(n_clusters=n_centroids)
kmeans.fit(data.cpu().numpy(), sample_weight=weights.cpu().numpy())
centroids = torch.from_numpy(kmeans.cluster_centers_).to(torch.float32)
d_distances = torch.cdist(data, centroids, p=2)**2
mu = torch.zeros(data.shape[0], n_centroids)
for i in range(data.shape[0]):
    mu[i, kmeans.labels_[i]] = 1
p = torch.einsum('ij, i -> j', mu, weights) / torch.sum(weights, dim=0)
s = torch.einsum('ij, i -> j', mu**2, weights) / torch.einsum('ij, i -> j', mu, weights)
E = torch.einsum('ij, i -> j', mu**2 * d_distances, weights) / torch.einsum('ij, i -> j', mu**2, weights)
measure = s * E ** (n_dimensions/2)
p_A = torch.einsum('ij, i -> j', mu[:data_A.shape[0], :], weights[:data_A.shape[0]]) / torch.sum(weights[:data_A.shape[0]])
p_B = torch.einsum('ij, i -> j', mu[data_A.shape[0]:, :], weights[data_A.shape[0]:]) / torch.sum(weights[data_A.shape[0]:])
value = (p_A - p_B) / (p_A + p_B)
integral = torch.sum(measure * value**2) / torch.sum(measure)
c_A = torch.max(mu[:data_A.shape[0], :], dim=0)[0]
c_B = torch.max(mu[data_A.shape[0]:, :], dim=0)[0]
card_inter = torch.sum(torch.minimum(c_A, c_B))
card_union = torch.sum(torch.maximum(c_A, c_B))
Jaccard_index = card_inter / card_union
distance = (1 + Jaccard_index)**(-1) * integral
print(f"{distance=}")
print(f"\t{Jaccard_index=}")
print(f"\t\t{card_inter=}", f"{card_union=}")
print(f"\t\t\t{c_A=}")
print(f"\t\t\t{c_B=}")
print(f"\t{integral=}")
print(f"\t\t{value=}")
print(f"\t\t\t{p_A=}")
print(f"\t\t\t{p_B=}")
print()
print()